In [39]:
import numpy as np
import math
from activation_func import activation_func as af
from tabulate import tabulate

In [40]:
class LSTM:
    def __init__(self, x, W_f, W_i, W_c, W_o, W_R, b_f, b_i, b_c, b_o, c_0, h_0, decimal_points = 3):
        self.x = x
        self.W_f = W_f
        self.W_i = W_i
        self.W_c = W_c
        self.W_o = W_o
        self.W_R = W_R
        self.b_f = b_f
        self.b_i = b_i
        self.b_c = b_c
        self.b_o = b_o
        self.c_0 = c_0
        self.h_0 = h_0
        self.dp = decimal_points

    def f_t(self, x, h):
        return af.binary_sigmoid(self.W_R @ h + self.W_f @ x + self.b_f)
    
    def i_t(self, x, h):
        return af.binary_sigmoid(self.W_R @ h + self.W_i @ x + self.b_i)
    
    def C_hat_t(self, x, h):
        return np.tanh(self.W_R @ h + self.W_c @ x + self.b_c)
    
    def C_t(self, f_t, c_t, i_t, c_hat_t):
        return (np.multiply(f_t, c_t) + np.multiply(i_t, c_hat_t))
    
    def o_t(self, x, h):
        return af.binary_sigmoid(self.W_R @ h + self.W_o @ x + self.b_o)
    
    def run(self, time_steps):
        iterations = time_steps if time_steps != None else len(self.x)
        table_1 = []
        table_2 = []
        ht = self.h_0
        ct = self.c_0
        for i in range(iterations):
            x_t = self.x[0] if time_steps != None else self.x[i]
            ft = self.f_t(x_t, ht)
            it = self.i_t(x_t, ht)
            c_hat_t = self.C_hat_t(x_t, ht)
            ct = self.C_t(ft, ct, it, c_hat_t)
            ot = self.o_t(x_t, ht)
            ht = np.multiply(ot, np.tanh(ct))
            table_1.append([i+1, x_t, np.round(ft, self.dp), np.round(it, self.dp), np.round(c_hat_t, self.dp)])
            table_2.append([np.round(ct, self.dp), np.round(ot, self.dp), np.round(ht, self.dp)])

        print(tabulate(table_1, headers=["t", "x_t", "f_t", "i_t", "C_hat_t"]))
        print(tabulate(table_2, headers=["C_t", "O_t", "h_t"]))



In [41]:
x = [[1, -1, 0, 1]]
W_f = np.array([[0, -1, 0, 1],
                [0, 1, -1, 1],
                [-1, 0, 1, 1]])

W_i = np.array([[-1, -1, -1, 0],
                [1, -1, 0, 1],
                [0, -1, 1, -1]])

W_c = np.array([[0, 1, -1, 1],
                [-1, 1, 1, 0],
                [1, -1, 0, -1]])

W_o = np.array([[1, 0, 1, -1],
                [-1, 1, 0, 1],
                [0, -1, 1, -1]])

W_R = np.array([[1, 0, -1],
                [-1, -1, 0],
                [0, -1, 1]])

b_f = [1, 0, -1]
b_i = [0, 0, -1]
b_c = [1, 1, 0]
b_o = [0, 1, -1]

h_0 = np.zeros(3)
c_0 = np.zeros(3)

lstm = LSTM(x, W_f, W_i, W_c, W_o, W_R, b_f, b_i, b_c, b_o, c_0, h_0)
lstm.run(time_steps=3)


  t  x_t            f_t                  i_t                  C_hat_t
---  -------------  -------------------  -------------------  ----------------------
  1  [1, -1, 0, 1]  [0.953 0.5   0.269]  [0.5   0.953 0.269]  [ 0.762 -0.762  0.762]
  2  [1, -1, 0, 1]  [0.958 0.532 0.346]  [0.532 0.958 0.346]  [ 0.81  -0.702  0.877]
  3  [1, -1, 0, 1]  [0.962 0.516 0.387]  [0.557 0.955 0.387]  [ 0.842 -0.732  0.912]
C_t                     O_t                  h_t
----------------------  -------------------  ----------------------
[ 0.381 -0.725  0.205]  [0.5   0.5   0.269]  [ 0.182 -0.31   0.054]
[ 0.796 -1.059  0.375]  [0.532 0.532 0.346]  [ 0.352 -0.418  0.124]
[ 1.234 -1.247  0.499]  [0.557 0.516 0.387]  [ 0.47  -0.438  0.179]
